In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('image', cmap='Blues')
from tools.IonChainTools import *
from scipy import fft

In [ ]:
target_size = (1200,1920) # the number of pixels of slm
pixel_size = np.array([7.8e-6,7.8e-6]) # the size of one pixel of camera
ions = np.array([0,1,2,5]) # the target ion

In [ ]:
N = 6 #number of total ions
v = 2*π*1e6 #trap frequency
slm_size = target_size

#Create a virtual SLM and camera
from slmsuite.hardware.slms.slm import SLM
from slmsuite.hardware.cameras.camera import Camera
from slmsuite.hardware.cameraslms import FourierSLM

wav_um = 0.730
slm = SLM (slm_size[1],slm_size[0], dx_um=7.8, dy_um=7.8, wav_um=wav_um)
camera = Camera(target_size[1],target_size[0])

slm.set_measured_amplitude_analytic(800) #set the beam size of the SLM
fs = FourierSLM(camera, slm)

In [ ]:
def ion_position_act(N,v): #N is the number of the ions, v is the trap frequency
    return calcPositions(N)*lengthScale(v)

def ion_position_cam(N,v,pixel_size,magnititude = 200): 
    '''
    magnify the distance between the ions by 'magnitude' times
    return the pixel location in the camera
    '''
    position = ion_position_act(N,v)*magnititude//pixel_size[0]
    return position

In [ ]:
#calculate the location of ions in the fourier plane
mag = 100 * 7.8 / 2.2
position = np.array(ion_position_cam(N,v,pixel_size,magnititude= mag))
dy = target_size[0]//8 
pos = position[ions]
position_cam = np.column_stack((pos+target_size[1]//2,np.full(pos.shape,target_size[0]//2+dy))) 

#calculate the blaze grating phase
x_coordinates, y_coordinates = np.meshgrid(np.arange(slm_size[1]), np.arange(slm_size[0]))
(center_x, center_y) = (slm_size[1]//2,slm_size[0]//2)
field_array = np.zeros(slm_size, dtype=complex)
for position1 in position_cam:
    (x, y) = position1
    shift_x,shift_y = x - center_x,y- center_y
    fx,fy = shift_x/slm_size[1], shift_y/slm_size[0]
    blaze_grating_phase = 2 * np.pi * (fx * x_coordinates  + fy * y_coordinates)
    field = np.exp(1j * blaze_grating_phase)
    field_array += field
blaze_phase = np.angle(field_array)
# the nearfield on the SLM
nearfield = slm._get_measured_amplitude() * np.exp(1j * blaze_phase)
# the farfield on the SLM
farfield = fft.fftshift(fft.fft2(fft.fftshift(nearfield), norm="ortho"))

In [ ]:
#plot the enhanced amplitude 
am_ff = np.abs(farfield)
# norm_amp = (am_ff - am_ff.min())/(am_ff.max() - am_ff.min())
log_scale_data = np.log(1+np.sqrt(am_ff))
from matplotlib import ticker
plt.figure(figsize=(8, 6))
img = plt.imshow(log_scale_data)
cb = plt.colorbar(img, label='Amplitude', format='%.1f')

# Adjust the colorbar labels to show 10^log_data format
cb.formatter = ticker.FuncFormatter(lambda x, pos: f'10^{x:.1f}')
cb.update_ticks()

plt.title('Farfield Data Visualization')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()

In [ ]:
#calculate the intensity at position of ions
intensity = am_ff **2
cal = []
all = range(N)
pos_all = position[all]
position_all = np.column_stack((np.full(pos_all.shape,target_size[0]//2+dy),pos_all+target_size[1]//2))
for i, pos in enumerate(position_all):
    cal.append(np.sum(intensity[int(pos[0])-60:int(pos[0])+60,int(pos[1])-60:int(pos[1])+60]))

In [ ]:
#plot the intensity for different ions
plt.scatter(range(6),cal, c='blue', marker='o', label='data')
plt.xlabel('ions')
plt.ylabel('intensity')
# plt.ylim(1250, 2250)
# plt.yticks(np.arange(1250,2250,125))
plt.legend()
plt.show()

In [ ]:
#calculate the fidelity
intensity = cal
background = 0

inten = np.array([ i- background for i in intensity])
all = range(N)
undesire = [ i for i in all if i not in ions]
Omega_desire = np.sqrt(inten[ions])
Omega_undesire = np.sqrt(inten[undesire])
F1 = sum(Omega_desire)/(len(ions)*max(Omega_desire))
F2 = 1-sum(Omega_undesire)/(len(undesire)*max(Omega_desire))
F = F1*F2

print(F1)
print(F2)
print(F)